In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import AdamW
import pandas as pd
import numpy as np
from WordPieceTokenizer import WordPieceTokenizer as Tokenizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm


dataFilePath = 'datasets/'
saveFilePath = 'saves/'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = Tokenizer(f'{dataFilePath}sentiment_vocab/vocab.txt',do_lower_case=False,strip_accents=False,clean_text=False)

In [2]:
df = pd.read_csv(f'{dataFilePath}sentiment_train.csv',index_col=0)
df.head()

,발화,감정,str_len,attention_mask,token_type_ids
0,2 2383 2354 2003 10368 1094 2706 16474 1015 3 ...,불안,24,1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,2 2092 163 2420 18281 2154 3 0 0 0 0 0 0 0 0 0...,불안,12,1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,2 2106 4701 1076 18939 1007 5406 1252 3 0 0 0 ...,불안,14,1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,2 10103 21602 2671 18048 2603 3 0 0 0 0 0 0 0 ...,불안,13,1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,2 2255 2913 11078 1111 1209 2677 3 0 0 0 0 0 0...,불안,11,1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [3]:
df.loc[(df['감정'] == '불안'),'감정'] = 0
df.loc[(df['감정'] == '당황'),'감정'] = 1
df.loc[(df['감정'] == '분노'),'감정'] = 2
df.loc[(df['감정'] == '슬픔'),'감정'] = 3
df.loc[(df['감정'] == '중립'),'감정'] = 4
df.loc[(df['감정'] == '행복'),'감정'] = 5
df.loc[(df['감정'] == '혐오'),'감정'] = 6

In [4]:
def process_dataframe(data_frame, device,batch_size,shuffle=False):
    tensor_x_list = []
    attentions = []
    for i in tqdm(range(len(data_frame))):
        token = data_frame.iloc[i,0]
        token = token.split(" ")
        token_list = []
        for t in token:
            token_list.append(int(t))
        tensor_x_list.append(token_list)
        
        attention = data_frame.iloc[i,3]
        attention = attention.split(" ")
        attention_list = []
        for a in attention:
            attention_list.append(int(a))
        attentions.append(attention_list)

    tensor_x = torch.tensor(tensor_x_list, dtype=torch.long, device=device)
    tensor_attention = torch.tensor(attentions, dtype=torch.long, device=device)
    tensor_t = torch.tensor(data_frame["감정"].values.tolist(), dtype=torch.long, device=device)

    dataset = TensorDataset(tensor_x,tensor_attention,tensor_t)

    loader = DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=True)
    
    return loader

In [5]:
train_df, val_df = train_test_split(df,train_size=0.8,test_size=0.2)

print(f"학습 세트의 크기: {len(train_df)} 행")
print(f"검증 세트의 크기: {len(val_df)} 행")

train_loader = process_dataframe(train_df,device,100,True)
val_loader = process_dataframe(val_df,device,1000,False)

학습 세트의 크기: 117078 행
검증 세트의 크기: 29270 행


100%|██████████████████████████████████████████████████████████████████████████| 29270/29270 [00:02<00:00, 9894.48it/s]


In [6]:
from Model import LSTM
from Model import Transformer, PositionalEncoding
vocab_size = tokenizer.get_vocab_size()
embedding_dim = 250

## LSTM

In [7]:
def LSTM_Train(epoch,device,train_loader,val_loader,NN,loss_function,optimizer):
    acc = 0
    prev_acc = 0
    cnt = 0
    for e in range(epoch):
        NN.to(device)
        loss_sum = 0
        NN.train()
        for x, attention,t in train_loader:
            y = NN(x,attention)
            loss = loss_function(y,t)
            loss_sum += loss.item()
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        loss_sum /= len(train_loader)
    
        NN.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x, attention, t in val_loader:
                x = x.to(device)
                attention = attention.to(device)
                t = t.to(device)
    
                y = NN(x, attention)
                correct += (y.argmax(dim=-1) == t).sum().item()
                total += len(x)
        acc = correct / total
    
        if acc <= prev_acc:
            cnt += 1
        else :
            torch.save(NN.state_dict(), "Sentiment.pt")
            cnt = 0
            prev_acc = acc
        
        print(f"epoch  {e+1}\t\tloss {loss_sum:.12f}\tacc {acc:.4f}\tcnt {cnt}")
        
        if cnt >= 5:
            print("train halted")
            break
            
    print("---------- 학습 종료 ----------")

In [8]:
# NN = LSTM(vocab_size=vocab_size,embedding_dim=embedding_dim,hidden_dim=64,output_dim=7,n_layers=4,bidirectional=True,dropout_p=0.1)
# NN.to(device)
# loss_function = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(NN.parameters(),lr=0.001)
# epoch = 500
# LSTM_Train(epoch,device,train_loader,val_loader,NN,loss_function,optimizer)

## Transformer

In [9]:
def Transformer_Train(epoch, device, train_loader, val_loader, NN, loss_function, optimizer,scheduler):
    acc = 0
    prev_acc = 0
    cnt = 0
    for e in range(epoch):
        NN.to(device)
        loss_sum = 0
        NN.train()
        for x, attention, t in tqdm(train_loader, desc=f"Epoch {e+1} Training",leave=False):
            x = x.to(device)
            attention = attention.to(device)
            t = t.to(device)

            y = NN(x, attention)
            loss = loss_function(y, t)
            loss_sum += loss.item()
        
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm(NN.parameters(),1.0)
            optimizer.step()
        loss_sum /= len(train_loader)
        
        NN.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x, attention, t in tqdm(val_loader, desc=f"Epoch {e+1} Validation",leave=False):
                x = x.to(device)
                attention = attention.to(device)
                t = t.to(device)
        
                y = NN(x, attention)
                correct += (y.argmax(dim=-1) == t).sum().item()
                total += len(x)
        acc = correct / total
        
        if acc <= prev_acc:
            cnt += 1
        else :
            torch.save(NN.state_dict(), "Sentiment.pt")
            cnt = 0
            prev_acc = acc

        scheduler.step(acc)
        
        print(f"epoch   {e+1}\t\tloss {loss_sum:.12f}\tacc {acc:.4f}\tcnt {cnt}")
        
        if cnt >= 5:
            print("train halted")
            break
            
    print("---------- 학습 종료 ----------")

In [11]:
NN = Transformer(vocab_size=vocab_size,embedding_dim=128,hidden_dim=16,output_dim=7,n_layers=2,n_heads=4,dropout_p=0.05,max_len=150,pad_token_id=0)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(NN.parameters(),lr=5e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode="max",factor=0.5,patience=3,)
epoch = 500
Transformer_Train(epoch,device,train_loader,val_loader,NN,loss_function,optimizer,scheduler)

Epoch 1 Training:   0%|                                                                       | 0/1170 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_24684\2332016856.py:20: FutureWarning: `torch.nn.utils.clip_grad_norm` is now deprecated in favor of `torch.nn.utils.clip_grad_norm_`.
  nn.utils.clip_grad_norm(NN.parameters(),1.0)
Epoch 1 Validation:   0%|                                                                       | 0/29 [00:00<?, ?it/s]C:\Users\Administrator\anaconda3\envs\tensor\Lib\site-packages\torch\nn\modules\transformer.py:505: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\NestedTensorImpl.cpp:182.)
  output = torch._nested_t

epoch   1		loss 1.611574331308	acc 0.4478	cnt 0


epoch   2		loss 1.521470283749	acc 0.4758	cnt 0


epoch   3		loss 1.440676615279	acc 0.5001	cnt 0


epoch   4		loss 1.392663119186	acc 0.5133	cnt 0


...

epoch   8		loss 1.282142842020	acc 0.5440	cnt 0


epoch   9		loss 1.262910174254	acc 0.5495	cnt 0


epoch   10		loss 1.244619312144	acc 0.5542	cnt 0


epoch   11		loss 1.227673865129	acc 0.5587	cnt 0


KeyboardInterrupt: 

In [ ]:
# model = torch.load("Sentiment.pt",weights_only=False)
# torch.save(model,f"{saveFilePath}train_15.pt")